In [2]:
import os 
from dotenv import load_dotenv
load_dotenv()


True

In [3]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [4]:
@tool
def multiply(a: int, b:int) -> int:
    """Give 2 Numbers a and b and returns their product"""
    return a*b

In [5]:
print(multiply.invoke({'a':3, 'b':3}))

9


In [6]:
multiply.name

'multiply'

In [7]:
multiply.description

'Give 2 Numbers a and b and returns their product'

In [8]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [9]:
# tool Calling

In [10]:
llm = ChatGroq(
    model="openai/gpt-oss-120b",
    api_key=os.getenv("GROQ_API_KEY"),
    temperature=0.8
)

In [11]:
llm_with_tools = llm.bind_tools([multiply]) # LLM mein Bohot saare Tools Bind Kr Skte.

### Tool Calling

In [12]:
llm_with_tools.invoke("Hi How are you?")

AIMessage(content="Hello! I'm doing great, thank you for asking. How can I assist you today?", additional_kwargs={'reasoning_content': 'The user says "Hi How are you?" Just a greeting, respond politely. No need for function.'}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 133, 'total_tokens': 182, 'completion_time': 0.102104013, 'prompt_time': 0.005593843, 'queue_time': 0.055224677, 'total_time': 0.107697856, 'completion_tokens_details': {'reasoning_tokens': 22}}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_626f3fc5e0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--2ccbe828-d19a-4770-b12f-655889b34c3d-0', usage_metadata={'input_tokens': 133, 'output_tokens': 49, 'total_tokens': 182})

In [40]:
query = HumanMessage("Can you multiply 3 with 1000")

In [41]:
messages = [query]

In [42]:
messages

[HumanMessage(content='Can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [43]:
result = llm_with_tools.invoke(messages)

In [44]:
result

AIMessage(content='', additional_kwargs={'reasoning_content': 'User wants multiplication 3 * 1000 = 3000. Use function multiply.', 'tool_calls': [{'id': 'fc_5b3e2336-c7d5-4584-bf9e-018d79aa777a', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 137, 'total_tokens': 190, 'completion_time': 0.112685613, 'prompt_time': 0.005417621, 'queue_time': 0.055072019, 'total_time': 0.118103234, 'completion_tokens_details': {'reasoning_tokens': 19}}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_a09bde29de', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--00bc6d30-dd9f-42b7-b29d-7d33ff522814-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'fc_5b3e2336-c7d5-4584-bf9e-018d79aa777a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 137, 'output_tokens': 53, 'total_tokens':

In [45]:
messages.append(result)

In [46]:
messages

[HumanMessage(content='Can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'User wants multiplication 3 * 1000 = 3000. Use function multiply.', 'tool_calls': [{'id': 'fc_5b3e2336-c7d5-4584-bf9e-018d79aa777a', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 137, 'total_tokens': 190, 'completion_time': 0.112685613, 'prompt_time': 0.005417621, 'queue_time': 0.055072019, 'total_time': 0.118103234, 'completion_tokens_details': {'reasoning_tokens': 19}}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_a09bde29de', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--00bc6d30-dd9f-42b7-b29d-7d33ff522814-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'fc_5b3e2336-c7d5-4584-bf9e-018d79aa777

In [47]:
result.tool_calls[0]['args']

{'a': 3, 'b': 1000}

In [48]:
llm_with_tools.invoke("Can you Multiply 3 with 10").tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 10},
  'id': 'fc_c710cb49-2fba-4762-8ed8-c3c38585ba21',
  'type': 'tool_call'}]

In [49]:
llm_with_tools.invoke("Can you Multiply 3 with 10").tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3, 'b': 10},
 'id': 'fc_b1c003e6-696c-49d7-bc78-597911503d92',
 'type': 'tool_call'}

In [50]:
# Tools ka Exceution LLMs Nahi Karte.

### Tool Execution

In [51]:
multiply.invoke(result.tool_calls[0]['args'])

3000

In [52]:
tool_result = multiply.invoke(result.tool_calls[0])

In [53]:
messages.append(tool_result)

In [54]:
messages

[HumanMessage(content='Can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'User wants multiplication 3 * 1000 = 3000. Use function multiply.', 'tool_calls': [{'id': 'fc_5b3e2336-c7d5-4584-bf9e-018d79aa777a', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 137, 'total_tokens': 190, 'completion_time': 0.112685613, 'prompt_time': 0.005417621, 'queue_time': 0.055072019, 'total_time': 0.118103234, 'completion_tokens_details': {'reasoning_tokens': 19}}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_a09bde29de', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--00bc6d30-dd9f-42b7-b29d-7d33ff522814-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'fc_5b3e2336-c7d5-4584-bf9e-018d79aa777

In [55]:
result.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3, 'b': 1000},
 'id': 'fc_5b3e2336-c7d5-4584-bf9e-018d79aa777a',
 'type': 'tool_call'}

In [56]:
result.tool_calls[0]['args']

{'a': 3, 'b': 1000}

In [57]:
multiply.invoke(result.tool_calls[0]['args'])

3000

In [62]:

multiply.invoke(result.tool_calls[0])

ToolMessage(content='3000', name='multiply', tool_call_id='fc_5b3e2336-c7d5-4584-bf9e-018d79aa777a')

In [59]:
multiply.invoke({'name': 'multiply',
 'args': {'a': 3, 'b': 10},
 'id': 'fc_18da9223-528a-41b0-a316-c58daa313e75',
 'type': 'tool_call'})

ToolMessage(content='30', name='multiply', tool_call_id='fc_18da9223-528a-41b0-a316-c58daa313e75')

In [66]:
llm_with_tools.invoke(messages)

AIMessage(content='The product of 3 and 1000 is **3000**.', additional_kwargs={'reasoning_content': 'We need to respond with the result.'}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 170, 'total_tokens': 203, 'completion_time': 0.068894555, 'prompt_time': 0.007817053, 'queue_time': 0.055246737, 'total_time': 0.076711608, 'completion_tokens_details': {'reasoning_tokens': 9}}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_626f3fc5e0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--0f39644f-e3de-4e58-ae0c-ebed668ccfd9-0', usage_metadata={'input_tokens': 170, 'output_tokens': 33, 'total_tokens': 203})

In [65]:
llm_with_tools.invoke(messages).content

'The product of 3 and 1000 is **3000**.'